In [9]:
import requests
import PySimpleGUI as sg

# Function to get currency exchange rates from Fixer API
def get_exchange_rate(api_key, base_currency, target_currency):
    url = f"http://data.fixer.io/api/latest?access_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data.get("success"):
            rates = data.get("rates", {})
            if base_currency in rates and target_currency in rates:
                base_rate = rates[base_currency]
                target_rate = rates[target_currency]
                return target_rate / base_rate
            else:
                sg.popup_error("Invalid currency codes!")
                return None
        else:
            sg.popup_error("API error:", data.get("error", {}).get("info", "Unknown error"))
            return None
    else:
        sg.popup_error("Failed to fetch exchange rates. Check your internet connection.")
        return None

# Main GUI function
def currency_converter():
    sg.theme("LightBlue")
    
    layout = [
        [sg.Text("Currency Converter", font=("Helvetica", 16), justification="center", expand_x=True)],
        [sg.Text("Source Currency:"), sg.InputText(key="source_currency", size=(10, 1))],
        [sg.Text("Destination Currency:"), sg.InputText(key="target_currency", size=(10, 1))],
        [sg.Text("Amount:"), sg.InputText(key="amount", size=(10, 1))],
        [sg.Button("Convert"), sg.Button("Exit")],
        [sg.Text("Converted Amount:"), sg.Text("", size=(15, 1), key="result", font=("Helvetica", 12))]
    ]
    
    window = sg.Window("Currency Converter", layout)
    api_key = "716aaf4639c41fdf3daf84d2576880a8"  # Replace with your Fixer API key

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, "Exit"):
            break
        elif event == "Convert":
            try:
                source_currency = values["source_currency"].upper()
                target_currency = values["target_currency"].upper()
                amount = float(values["amount"])
                
                rate = get_exchange_rate(api_key, source_currency, target_currency)
                if rate is not None:
                    converted_amount = amount * rate
                    window["result"].update(f"{converted_amount:.2f}")
            except ValueError:
                sg.popup_error("Please enter valid numeric values for the amount.")
    
    window.close()

# Run the application
if __name__ == "__main__":
    currency_converter()
